In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN','BRK-B','TSLA','NVDA','KMX','F']  #

data = yf.download(tickers, start="2020-01-01", end="2023-01-01")['Close']

data.head()

In [ ]:
returns = data.pct_change().dropna()
returns.head()

In [ ]:

mean_returns = returns.mean()

cov_matrix = returns.cov()

In [ ]:

risk_free_rate = 0.02

sharpe_ratio_threshold = 0.7

# Simulate portfolios
num_portfolios = 10000
results = np.zeros((3, num_portfolios))
weights_record = []

underperforming_count = 0
top_performing_portfolios = []

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)  # Normalize the weights so they add up to 1
    weights_record.append(weights)

    # Calculate the portfolio's expected return and volatility
    portfolio_return = np.dot(weights, mean_returns) * 252  # Annualize return
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 252, weights)))  # Annualize volatility

    # Adjust the Sharpe ratio calculation with the risk-free rate
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility

    # Save the results for plotting
    results[0, i] = portfolio_return
    results[1, i] = portfolio_volatility
    results[2, i] = sharpe_ratio

    # Check if the portfolio is underperforming
    if sharpe_ratio < sharpe_ratio_threshold:
        underperforming_count += 1

    # Store the portfolio number (i), return, volatility, Sharpe ratio, and weights for sorting later
    top_performing_portfolios.append((i, portfolio_return, portfolio_volatility, sharpe_ratio, weights))

# Sort the portfolios by Sharpe ratio in descending order to get the best performers
top_performing_portfolios = sorted(top_performing_portfolios, key=lambda x: x[3], reverse=True)

# Get the top 5 best-performing portfolios
top_5_portfolios = top_performing_portfolios[:5]

# Display the results
print(f"Number of underperforming portfolios: {underperforming_count}")

print("\nTop 5 best-performing portfolios:")
for idx, (portfolio_index, portfolio_return, portfolio_volatility, sharpe_ratio, weights) in enumerate(top_5_portfolios):
    print(f"\nPortfolio {portfolio_index + 1}:")  # Portfolio number keeps its original index (e.g., 3605)
    print(f"  Return: {portfolio_return:.2f}%")
    print(f"  Volatility: {portfolio_volatility:.2f}%")
    print(f"  Sharpe Ratio: {sharpe_ratio:.2f}")

    # Show the associated stocks with their weights and percentage
    print("  Stocks and their weights (%):")
    for i, weight in enumerate(weights):
        print(f"    {tickers[i]}: {weight * 100:.2f}%")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.5)
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.5)

# Find the portfolio with the highest Sharpe ratio
best_sharpe_idx = np.argmax(results[2])

# Highlight the portfolio with the highest Sharpe ratio
plt.scatter(results[1][best_sharpe_idx], results[0][best_sharpe_idx], color='red', marker='*', s=200, label='Best Portfolio')

# Print the Sharpe ratio of the best portfolio
best_sharpe_ratio = results[2][best_sharpe_idx]
print(f"The Sharpe Ratio of the Best Portfolio is: {best_sharpe_ratio:.4f}")

plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier')
plt.grid(True)
plt.legend(loc='upper left')
plt.show()

In [ ]:
max_sharpe_idx = np.argmax(results[2])

# Get the optimal weights corresponding to that portfolio
optimal_weights = weights_record[max_sharpe_idx]

# Print the best portfolio allocation (max Sharpe ratio)
print("Best Portfolio Allocation (Max Sharpe Ratio):")
for i, ticker in enumerate(tickers):
    print(f"{ticker}: {optimal_weights[i]*100:.2f}%")